In [ ]:
!pip install datasets
from tokenizers import (decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer)
from transformers import GPT2Tokenizer, GPT2TokenizerFast, GPT2Model, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from huggingface_hub import login
import logging
import time
from datasets import Dataset, DatasetDict
import os
import torch


In [ ]:
# Вход в Hugging Face Token
hf_token='XXXXXXXX'
login(token=hf_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


# New tokenizer + merging

In [ ]:

# new tokenizer

new_tokenizer = Tokenizer(models.BPE())
new_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
trainer = trainers.BpeTrainer(vocab_size=5000, special_tokens=["<|endoftext|>"])
train_file = '/content/drive/MyDrive/Study/train_ext_file.txt'
new_tokenizer.train([train_file], trainer=trainer)
new_tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
new_tokenizer.decoder = decoders.ByteLevel()

new_tokenizer = GPT2TokenizerFast(tokenizer_object=new_tokenizer)
new_tokenizer.save_pretrained("/content/drive/MyDrive/Study/new_tokenizer_gpt2")
new_tokenizer

# gpt2 tokenizer
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
print(len(gpt2_tokenizer.get_vocab()))
gpt2_tokenizer

# merge the vocabulary for the extended tokenizer
vocab_tokens = list(new_tokenizer.get_vocab())
decoded_tokens = [new_tokenizer.decoder.decode([token]) for token in vocab_tokens]
print(len(vocab_tokens), len(decoded_tokens))
gpt2_tokenizer.add_tokens(decoded_tokens)
gpt2_tokenizer.save_pretrained("/content/drive/MyDrive/Study/extended_tokenizer_gpt2")
print(len(gpt2_tokenizer.get_vocab()))
gpt2_tokenizer

# validate the changes
text = "Бутерброды с сыром"
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
new_tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/Study/new_tokenizer_gpt2")
extended_tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/Study/extended_tokenizer_gpt2")

print(len(gpt2_tokenizer.encode(text)))
print(gpt2_tokenizer.encode(text))
print(len(new_tokenizer.encode(text)))
print(new_tokenizer.encode(text))
print(len(extended_tokenizer.encode(text)))
print(extended_tokenizer.encode(text))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

50257
5000 5000
54801


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


39
[140, 239, 35072, 20375, 16843, 21169, 140, 109, 21169, 25443, 112, 45035, 220, 21727, 220, 21727, 45035, 21169, 25443, 120, 12466, 110, 45035, 141, 230, 30143, 18849, 12466, 123, 21169, 25443, 111, 35072, 30143, 40623, 20375, 45367, 21727, 40623]
14
[850, 375, 921, 744, 205, 215, 1165, 645, 2072, 311, 3406, 1164, 672, 277]
14
[54681, 51802, 50310, 54339, 51801, 53868, 51606, 53775, 52577, 54504, 54759, 53798, 54360, 54682]


# Preparing Dataset

In [ ]:
#Preparing Dataset from your text file (skip this block if you have lm_dataset)


# Suppress the warning messages
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

# Load the tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/Study/extended_tokenizer_gpt2")
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.vocab_size)
print(len(tokenizer))


train_file = "/content/drive/MyDrive/Study/train_ext_file.txt"  # Replace with your training data file
valid_file = "/content/drive/MyDrive/Study/valid_ext_file.txt"  # Replace with your validation data file
# Read the text data from the .txt file
with open(train_file, 'r', encoding='utf-8') as f:
    train_data = f.readlines()
with open(valid_file, 'r', encoding='utf-8') as f:
    valid_data = f.readlines()

print(len(train_data), len(valid_data))

# Create a Dataset object from the text data
train_dataset = Dataset.from_dict({"text": train_data})
valid_dataset = Dataset.from_dict({"text": valid_data})

def preprocess_function(examples):
    out = tokenizer(examples["text"])
    # out = tokenizer([" ".join(x) for x in examples["text"]])
    return out

# Apply tokenization to the dataset in batches using map
train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc=4, remove_columns=train_dataset.column_names)
time.sleep(5)
valid_dataset = valid_dataset.map(preprocess_function, batched=True, num_proc=4, remove_columns=valid_dataset.column_names)
tokenized_datasets = DatasetDict({"train": train_dataset, "valid":valid_dataset})

# Group the tokenized datasets into blocks
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy() # First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.
    return result

# Group the tokenized train dataset
lm_train_dataset = tokenized_datasets['train'].map(group_texts, batched=True, num_proc=4)
time.sleep(2)
# Group the tokenized valid dataset
lm_valid_dataset = tokenized_datasets['valid'].map(group_texts, batched=True, num_proc=4)

lm_dataset = DatasetDict({"train": lm_train_dataset, "valid":lm_valid_dataset})
lm_dataset
# Сохранение датасета
save_path = "/content/drive/MyDrive/Study/lm_dataset"
lm_dataset.save_to_disk(save_path)
# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

out = data_collator([lm_dataset['train'][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

50257
54801
333611 16272


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/333611 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/16272 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/333611 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/16272 [00:00<?, ? examples/s]

input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [ ]:
#If Dataset is ready
from datasets import load_from_disk

# Загрузка датасета
load_path = "/content/drive/MyDrive/Study/lm_dataset"
lm_dataset = load_from_disk(load_path)


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f" Device: {device}")

#NEW MODEL
# Load the GPT-2 base model
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Resize the model's embeddings to match the vocabulary size of our tokenizer
def find_multiple(n: int, k: int) -> int:
    if n % k == 0:
        return n
    return n + k - (n % k)

new_embeddings_size = find_multiple(len(tokenizer), 64)
model.resize_token_embeddings(new_embeddings_size)

# Optionally, freeze certain layers of the model (e.g., except embeddings)
freeze_layers = False

if freeze_layers:
    for name, param in model.named_parameters():
        if 'transformer.wte' in name:
            param.requires_grad = True
        else:
            param.requires_grad = False

Используемое устройство: cuda


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Train

In [ ]:
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Study/rus_gpt2_2_epoch/",
    overwrite_output_dir=True,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="steps",
    eval_steps=2000,
    logging_steps=2000,
    gradient_accumulation_steps=1,
    num_train_epochs=2,
    weight_decay=0.1,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    learning_rate=5e-3,
    save_steps=10000,
    save_total_limit=2,
    push_to_hub=True,
    fp16=True,

)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=lm_train_dataset,
    eval_dataset=lm_valid_dataset,
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.save_model("/content/drive/MyDrive/Study/rus_gpt2_2_epoch")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
2000,4.211100,3.560251
4000,3.525900,3.341156
6000,3.362500,3.212029
8000,3.236700,3.099142
10000,3.101800,2.977775
12000,2.974300,2.849730
14000,2.851800,2.745262
16000,2.770500,2.701861


# Inference

In [ ]:
# Load the fine-tuned GPT-2 model for inference
tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/Study/extended_tokenizer_gpt2")
tokenizer.pad_token = tokenizer.eos_token
finetuned_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Study/rus_gpt2_2_epoch")

finetuned_model.eval()

# Input text for completion
input_text = "Однажды она решила"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")



max_length = 250
# # Generate text completions
# output_text = finetuned_model.generate(input_ids, max_length=max_length)[0]

output_text = finetuned_model.generate(
    input_ids,
    max_length=max_length,
    temperature=0.8,              # Controls the diversity of the generated text
    top_k=50,                     # Keeps only the top-k most likely words
    top_p=0.9,                    # Nucleus sampling (cumulative probability)
    repetition_penalty=1.2,       # Penalty for repeating words or phrases
    no_repeat_ngram_size=4,       # Prevents repetition of n-grams (e.g., bigrams)
    do_sample=True,                # Enables sampling for greater diversity
    pad_token_id=tokenizer.eos_token_id
)[0]

#Decode the generated token IDs to text
completed_text = tokenizer.decode(output_text, skip_special_tokens=True, clean_up_tokenization_spaces=True)


print("Input Text:", input_text)
print("Completed Text:", completed_text)